In [1]:
from evaluations2 import compute_kl, compute_mmd, compute_wasserstein, compute_random_forest

In [3]:
import numpy as np

def generate_dummy_data(n_samples: int = 100, seed: int = 42) -> tuple[np.ndarray, np.ndarray]:
    """
    Génère deux matrices NumPy (Original et Generated) de taille (n_samples, 3)
    avec des relations polynomiales et du bruit.

    Args:
        n_samples: Nombre d'observations (lignes).
        seed: Graine pour la reproductibilité.

    Returns:
        Un tuple contenant (Original_data, Generated_data).
    """
    np.random.seed(seed)

    # --- 1. Génération des données de base (Original) ---
    # Création d'une variable de base x (de 0 à 10 pour l'exemple)
    x = np.linspace(0, 10, n_samples)

    # Définition des 3 features (relations polynomiales)
    feature_1 = x                 # F1: Linéaire (x)
    feature_2 = x**2              # F2: Quadratique (x^2)
    feature_3 = x**3              # F3: Cubique (x^3)

    # Assemblage de la matrice Original (sans ordre)
    original_data_ordered = np.column_stack((feature_1, feature_2, feature_3))

    # Mélange des lignes (observations) pour casser l'ordre initial
    # np.random.permutation renvoie un nouvel ordre d'indices
    shuffle_indices = np.random.permutation(n_samples)
    Original = original_data_ordered[shuffle_indices]


    # --- 2. Génération des données bruitées (Generated) ---
    # On reprend les données Original (ordonnées) pour s'assurer que le bruit est appliqué
    # sur la même base
    Generated_base = original_data_ordered.copy()

    # Ajout du bruit (Gaussien) pour simuler la génération/transformation
    # L'échelle du bruit est ajustée pour être pertinente par rapport aux données
    scale_f1 = np.std(feature_1) * 0.3  # 10% de l'écart-type de F1
    scale_f2 = np.std(feature_2) * 0.4  # 10% de l'écart-type de F2
    scale_f3 = np.std(feature_3) * 0.1 # 5% de l'écart-type de F3 (car x^3 est plus grand)

    # Ajout du bruit différent pour chaque colonne
    noise_f1 = np.random.normal(0, scale_f1, n_samples)
    noise_f2 = np.random.normal(0, scale_f2, n_samples)
    noise_f3 = np.random.normal(0, scale_f3, n_samples)

    Generated_base[:, 0] += noise_f1
    Generated_base[:, 1] += noise_f2
    Generated_base[:, 2] += noise_f3

    # Mélange des lignes de Generated (en utilisant les mêmes indices pour la cohérence,
    # mais n'est pas strictement nécessaire pour la mesure des métriques)
    Generated = Generated_base[shuffle_indices]

    print(f"Original matrix shape: {Original.shape}")
    print(f"Generated matrix shape: {Generated.shape}")
    
    return Original, Generated

# --- Exécution ---
Original_data, Generated_data = generate_dummy_data(n_samples=100)

Original matrix shape: (100, 3)
Generated matrix shape: (100, 3)


In [4]:
compute_random_forest(
    original=Original_data,
    generated=Generated_data,
    output_path="./",
    figure_name="roc_curve.png")

OOB score of random forest: 0.5733333333333334
Accuracy in training set: 0.8933333333333333
Accuracy in validation set: 0.56
AUC value: 0.6864


{'train accuracy': 0.8933333333333333,
 'validation accuracy': 0.56,
 'auc': 0.6864000000000001,
 'oob_score': 0.5733333333333334,
 'False Positive Rate': [0.0,
  0.0,
  0.0,
  0.08,
  0.08,
  0.12,
  0.12,
  0.16,
  0.16,
  0.32,
  0.32,
  0.36,
  0.36,
  0.4,
  0.4,
  0.52,
  0.52,
  0.56,
  0.56,
  0.64,
  0.64,
  0.68,
  0.68,
  0.8,
  0.8,
  1.0],
 'True Positive Rate': [0.0,
  0.04,
  0.12,
  0.12,
  0.4,
  0.4,
  0.44,
  0.44,
  0.52,
  0.52,
  0.56,
  0.56,
  0.6,
  0.6,
  0.64,
  0.64,
  0.68,
  0.68,
  0.72,
  0.72,
  0.92,
  0.92,
  0.96,
  0.96,
  1.0,
  1.0]}

In [5]:
compute_kl(original=Original_data, generated=Generated_data)

array(0.00285076)

In [6]:
compute_wasserstein(original=Original_data, generated=Generated_data)

19.40526869043339

In [7]:
compute_mmd(original=Original_data, generated=Generated_data)

array(0.02836536)